In [1]:
#Segmenting and Clustering Neighbourhoods in Toronto Canada Part 1 
#web scrape data table, create pandas dataframe and clean up dataframe

# install beautifulsoup 4 and parser

! pip install beautifulsoup4
! pip install lxml
! pip install geocoder

     |████████████████████████████████| 102kB 9.6MB/s eta 0:00:01


In [2]:
# import libraries

import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

In [130]:
# get html from Wikipedia and parse the table

res =  requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content, 'lxml')
table = soup.find_all('table')[0]

#print(table)

In [131]:
#create a pandas dataframe from the parsed html table

df = pd.read_html(str(table))[0]
df.head(10)
df.shape


(287, 3)

In [132]:
#Clean the dataframe of Boroughs=Not assigned values, postcodes with multiple neigborhoods etc.

#drop rows with Borough = Not assigned
df.rename(columns={'District': 'Borough'}, inplace=True)
df.drop(df[df['Borough']=='Not assigned'].index, inplace=True)
#df.head(10)
df.shape



(210, 3)

In [6]:
#Combine postal code with multiple neighborhoods
#df.groupby(['Postcode','Borough'], as_index=False, sort=False).agg(lambda x:','.join(x))

df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(list)
df = df.sample(frac=1).reset_index()
df['Neighbourhood'] = df['Neighbourhood'].str.join(',')
df.head()



,Postcode,Borough,Neighbourhood
0,M6G,Downtown Toronto,Christie
1,M2N,North York,Willowdale South
2,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
3,M4L,East Toronto,"The Beaches West,India Bazaar"
4,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"


In [7]:
#Assign value of Borough to Neighbourhood when Neighbourhood is 'Not assigned'

df.loc[df['Neighbourhood'] == 'Not assigned', 'Neighbourhood'] = df['Borough']
df.head(11)

,Postcode,Borough,Neighbourhood
0,M6G,Downtown Toronto,Christie
1,M2N,North York,Willowdale South
2,M9V,Etobicoke,"Albion Gardens,Beaumond Heights,Humbergate,Jam..."
3,M4L,East Toronto,"The Beaches West,India Bazaar"
4,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
5,M1N,Scarborough,"Birch Cliff,Cliffside West"
6,M5A,Downtown Toronto,Harbourfront
7,M1E,Scarborough,"Guildwood,Morningside,West Hill"
8,M5B,Downtown Toronto,"Ryerson,Garden District"
9,M3J,North York,"Northwood Park,York University"


In [8]:
#Shape of dataframe after aggregating Neighbourhoods etc.
df.shape

#End Part1 of Segmenting and clustering Neighbourhoods in Toronto CA

(103, 3)

# Part 2: get the geographical coordinates (lat, long) for each postal code. 


In [9]:

#import geospatial_coordinate csv into dataframe


df_coor = pd.read_csv('https://cocl.us/Geospatial_data')
df_coor.rename(columns={'Postal Code':'Postcode'}, inplace=True)#rename the 1st column to match the original dataframe
df_coor.head(10)

,Postcode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [10]:
#Assign the geospatial coordinates to the postcodes using merge

df_spatial = pd.merge(df,df_coor, left_index=False, right_index=False, sort=True)
df_spatial.head(10)#verify the merge is correct(visually) between df_coor and df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


## Part 3 of Segmenting and Clustering Toronto: Visualize and analyze spatial data.
## Workflow will be similar to Lab-analysis of New York City data.

In [11]:

#download dependencies
!conda update -n base -c defaults conda
import json #library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values


# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library


Solving environment: done

## Package Plan ##

  environment location: /opt/conda

  added / updated specs: 
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    tqdm-4.40.2                |             py_0          53 KB
    conda-4.8.0                |           py37_1         3.0 MB
    conda-package-handling-1.6.0|   py37h7b6447c_0         872 KB
    ------------------------------------------------------------
                                           Total:         3.9 MB

The following NEW packages will be INSTALLED:

    conda-package-handling: 1.6.0-py37h7b6447c_0
    tqdm:                   4.40.2-py_0         

The following packages will be UPDATED:

    conda:                  4.5.12-py37_0        --> 4.8.0-py37_1


tqdm-4.40.2          | 53 KB     | ##################################### | 100% 
conda-4.8.0          | 3.0 MB    | ###################################

In [12]:
#define instance of geocoder, user_agent tor_explorer

address = 'Toronto, CA'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [129]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_spatial['Latitude'], df_spatial['Longitude'], df_spatial['Borough'], df_spatial['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
#Cluster and explore only the neighbourhoods in Toronto.

df_toronto = df_spatial['Borough'].isin(['West Toronto', 'Central Toronto', 'Downtown Toronto'])#use isin series method to get Toronto neighbourhood dataframe

df_toronto = df_spatial[df_toronto]

#df_toronto.shape
#df_toronto.head()

In [128]:
# create map of just Toronto using latitude and longitude values
map_tor = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_toronto['Latitude'], df_toronto['Longitude'],df_toronto['Borough'], df_toronto['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_tor)  
    
map_tor

In [16]:
#Define Foursquare credentials and Version

CLIENT_ID = 'JS013FPKBDGJRBSDFOCOALDNLLT2YKW1MJYTBTOLLV3LOVVH'
CLIENT_SECRET = '45B4TIRR1OTTLJN4CBE4SEZ5NNTZBMPELUWAN3KCJDRUYFF5'
VERSION = '20180605'



In [17]:
#define neighbourhood to explore
#df_toronto.head(30)
df_toronto.loc[55,'Neighbourhood']

'St. James Town'

In [18]:
#Get the neighbourhoods latitude and longitude


neighbourhood_latitude = df_toronto.loc[55, 'Latitude']
neighbourhood_longitude = df_toronto.loc[55,'Longitude']

neighbourhood_name = df_toronto.loc[55,'Neighbourhood']


In [19]:
#Get the top 100 venues that are in Rosedale within a radius of 500 meters.


LIMIT = 100 # number of venues returned by Foursquare API
radius = 500 # define radius

#create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighbourhood_latitude, 
    neighbourhood_longitude, 
    radius, 
    LIMIT)

In [163]:
#Send the GET request and examine results

results = requests.get(url).json()


In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
#Clean the json file and structure into pandas dataframe

venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Crepe TO,Creperie,43.650063,-79.374587
1,Gyu-Kaku Japanese BBQ,Japanese Restaurant,43.651422,-79.375047
2,Terroni,Italian Restaurant,43.650927,-79.375602
3,GoodLife Fitness Toronto 137 Yonge Street,Gym,43.651242,-79.378068
4,GEORGE Restaurant,Restaurant,43.653346,-79.374445


In [23]:
#print number of venues returned
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [24]:
#Function to get top 100 venues for each neighbourhood in Toronto

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [25]:
#create new dataframe called toronto_venues from function above

toronto_venues = getNearbyVenues(names = df_toronto['Neighbourhood'],
                                 latitudes = df_toronto['Latitude'],
                                 longitudes = df_toronto['Longitude']
                                )



Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede,Swansea
Queen's Park


In [26]:
#check resulting dataframe
print(toronto_venues.shape)
toronto_venues.head()


(1564, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Lawrence Park,43.728020,-79.388790,Lawrence Park Ravine,43.726963,-79.394382,Park
1,Lawrence Park,43.728020,-79.388790,Booty Camp Fitness,43.728051,-79.387853,Gym / Fitness Center
2,Lawrence Park,43.728020,-79.388790,Zodiac Swim School,43.728532,-79.382860,Swim School
3,Lawrence Park,43.728020,-79.388790,TTC Bus #162 - Lawrence-Donway,43.728026,-79.382805,Bus Line
4,Davisville North,43.712751,-79.390197,Sherwood Park,43.716551,-79.387776,Park


In [27]:
#check how many venues were returned for each neighbourhood

toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton,Exhibition Place,Parkdale Village",22,22,22,22,22,22
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown,Grange Park,Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17
Church and Wellesley,83,83,83,83,83,83


In [28]:
#find out how many unique categories can be curated from all the returned venues

print('There are {} unique categories.' .format(len(toronto_venues['Venue Category'].unique())))

There are 224 unique categories.


In [29]:
#Analysis of each neighborhood

# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

print(toronto_onehot.shape)
toronto_onehot.head()

(1564, 224)


,Yoga Studio,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [30]:
#Group rows by neighborhood by taking the mean and frequency of occurence of each category

toronto_grouped =toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.shape

(33, 224)

In [31]:
#Print each neighborhood along with the top 5 most common venues

num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2       Steakhouse  0.04
3  Thai Restaurant  0.04
4     Burger Joint  0.03


----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1        Beer Bar  0.04
2     Cheese Shop  0.04
3      Steakhouse  0.04
4  Farmers Market  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3     Yoga Studio  0.05
4          Bakery  0.05


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                 venue  freq
0     Airport Terminal  0.13
1       Airport Lounge  0.13
2      Airport Service  0.13
3  Rental Car Location  0.07
4     Sculpture Garden  0.07


----Cabbagetown,St. James Town----
         venue  freq
0  Coffee Shop  0.09
1   Restaurant  0.07
2         Café  0.05
3  Flower Shop  0.05
4       Bakery  0.05


----Ce

# Put the sorted neighorhoods with most common venues into a dataframe


In [32]:
#define function to sort the venues in descending order

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [143]:
#Create the new dataframe and display the top 10 venues for each neighborhood

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Steakhouse,Thai Restaurant,Bar,Bakery,Restaurant,Burger Joint,Salad Place,Asian Restaurant
1,Berczy Park,Coffee Shop,Farmers Market,Bakery,Seafood Restaurant,Cocktail Bar,Beer Bar,Cheese Shop,Steakhouse,Café,Irish Pub
2,"Brockton,Exhibition Place,Parkdale Village",Breakfast Spot,Café,Coffee Shop,Grocery Store,Gym,Restaurant,Pet Store,Performing Arts Venue,Italian Restaurant,Intersection
3,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Sculpture Garden,Airport,Airport Food Court,Airport Gate,Boutique,Bar
4,"Cabbagetown,St. James Town",Coffee Shop,Restaurant,Flower Shop,Pub,Italian Restaurant,Bakery,Café,Pizza Place,Grocery Store,Park


# Cluster Neighborhoods

In [144]:
#Run K means to cluster the neighborhoods into 5 clusters

# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [145]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_toronto

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood', how='right')

toronto_merged.head() # check the last columns!


,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,4,Park,Gym / Fitness Center,Swim School,Bus Line,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
45,M4P,Central Toronto,Davisville North,43.712751,-79.390197,0,Hotel,Clothing Store,Sandwich Place,Breakfast Spot,Pizza Place,Gym,Park,Food & Drink Shop,Doner Restaurant,Donut Shop
46,M4R,Central Toronto,North Toronto West,43.715383,-79.405678,0,Clothing Store,Coffee Shop,Sporting Goods Shop,Park,Chinese Restaurant,Cosmetics Shop,Mexican Restaurant,Burger Joint,Rental Car Location,Restaurant
47,M4S,Central Toronto,Davisville,43.704324,-79.388790,0,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Café,Gym,Sushi Restaurant,Seafood Restaurant,Park
48,M4T,Central Toronto,"Moore Park,Summerhill East",43.689574,-79.383160,1,Playground,Intersection,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [146]:
#Visualize the resulting clusters

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Examine the Clusters and determine the discriminating venue categories


In [159]:
#toronto_merged.loc[toronto_merged['Cluster Labels'] ==0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]
pd.crosstab(toronto_merged.Borough,toronto_merged['1st Most Common Venue'])

1st Most Common Venue,Airport Lounge,Bakery,Bar,Breakfast Spot,Café,Clothing Store,Coffee Shop,Gift Shop,Grocery Store,Health & Beauty Service,Hotel,Mexican Restaurant,Park,Playground,Sandwich Place,Sushi Restaurant
Borough,,,,,,,,,,,,,,,,
Central Toronto,0,0,0,0,1,1,1,0,0,1,1,0,2,1,1,0
Downtown Toronto,1,0,0,0,3,0,11,0,1,0,0,0,1,0,0,1
West Toronto,0,1,1,1,0,0,1,1,0,0,0,1,0,0,0,0


In [149]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]


,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
48,Central Toronto,1,Playground,Intersection,Restaurant,Wings Joint,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [150]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Central Toronto,2,Health & Beauty Service,Garden,Wings Joint,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [151]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
50,Downtown Toronto,3,Park,Trail,Playground,Dance Studio,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run,Discount Store
64,Central Toronto,3,Park,Trail,Sushi Restaurant,Jewelry Store,General Entertainment,Electronics Store,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


In [152]:
toronto_merged.loc[toronto_merged['Cluster Labels'] ==4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
44,Central Toronto,4,Park,Gym / Fitness Center,Swim School,Bus Line,Deli / Bodega,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


# The discriminating venue categories appear to be 1)Coffe shope/Cafe, 2) Parks&Trails, 3)Health,Beauty & Garden, 4)Playground &Intersection, 5) Park & Gym/Fitness.